In [1]:
import keras
import tensorflow
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.models import Sequential, Model 
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras_video import VideoFrameGenerator

import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
import keras_metrics as km
from keras import metrics
import glob
 
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import multilabel_confusion_matrix
from sklearn.metrics import classification_report


In [2]:
data_dir = "./videodata/"
filename = "ConvLSTM_model.h5"
h, w = 140, 90
time_steps = 10

#classes = ["Backstroke", "Breaststroke", "Freestyle"]
classes = [i.split(os.path.sep)[1] for i in glob.glob('./videodata/*')]
classes.sort()
#print(classes)

#glob_pattern='./videodata/{classname}/*.avi'

#data_aug = ImageDataGenerator(
#    zoom_range = [0.6, 1.2],
#    horizontal_flip = True,
#    rotation_range = 180,
#    width_shift_range = 0,
#    height_shift_range = 0)

#train = VideoFrameGenerator(
#    classes=classes, 
#    glob_pattern=glob_pattern,
#    nb_frames=time_steps,
#    split = 0.3, 
#    shuffle = True,
#    batch_size = 1,
#    target_shape = (w, h),
#    nb_channel = 3,
#    transformation=data_aug,
#    use_frame_cache=True)
#valid = train.get_validation_generator()

In [7]:
# Zoom in
def cv2_clipped_zoom(img, zoom_factor):
    """
    Center zoom in/out of the given image and returning an enlarged/shrinked view of 
    the image without changing dimensions
    Args:
        img : Image array
        zoom_factor : amount of zoom as a ratio (0 to Inf)
    """
    height, width = img.shape[:2] # It's also the final desired shape
    new_height, new_width = int(height * zoom_factor), int(width * zoom_factor)

    ### Crop only the part that will remain in the result (more efficient)
    # Centered bbox of the final desired size in resized (larger/smaller) image coordinates
    y1, x1 = max(0, new_height - height) // 2, max(0, new_width - width) // 2
    y2, x2 = y1 + height, x1 + width
    bbox = np.array([y1,x1,y2,x2])
    # Map back to original image coordinates
    bbox = (bbox / zoom_factor).astype(np.int)
    y1, x1, y2, x2 = bbox
    cropped_img = img[y1:y2, x1:x2]

    # Handle padding when downscaling
    resize_height, resize_width = min(new_height, height), min(new_width, width)
    pad_height1, pad_width1 = (height - resize_height) // 2, (width - resize_width) //2
    pad_height2, pad_width2 = (height - resize_height) - pad_height1, (width - resize_width) - pad_width1
    pad_spec = [(pad_height1, pad_height2), (pad_width1, pad_width2)] + [(0,0)] * (img.ndim - 2)

    result = cv2.resize(cropped_img, (resize_width, resize_height))
    result = np.pad(result, pad_spec, mode='constant')
    assert result.shape[0] == height and result.shape[1] == width
    return result

#  Creating frames from videos
def frames_extraction(video_path):
    frames = []
     
    video = cv2.VideoCapture(video_path)
    # Used as counter variable 
    count = 1
    while count <= time_steps:
        ret, img = video.read() 
        if ret:
            img = cv2.resize(img, (w, h))
            #img = cv2_clipped_zoom(img, 1.2)
            
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            #img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            
            frames.append(img)
            count += 1
        else:
            print("Defected frame")
            break
    #return frames
    rotate_frames = []
    zoom_frames = []
    for img in frames:
        rimg = cv2.rotate(img, cv2.ROTATE_180)
        rotate_frames.append(rimg)
    for img in frames:
        zimg = cv2_clipped_zoom(img, 1.5)
        zoom_frames.append(zimg)

        
    return frames, rotate_frames, zoom_frames

def create_data(input_dir):
    X = []
    Y = []
     
    classes_list = os.listdir(input_dir)
    print(classes_list)
    for c in classes_list:
        print(c)
        files_list = os.listdir(os.path.join(input_dir, c))
        for f in files_list:
            frames, rotate_frames, zoom_frames = frames_extraction(os.path.join(os.path.join(input_dir, c), f))
            #frames = frames_extraction(os.path.join(os.path.join(input_dir, c), f))
            
            #if len(frames) == time_steps:
            #    X.append(frames)
            #    y = [0]*len(classes)
            #    y[classes.index(c)] = 1
            #    Y.append(y)
                
            if len(frames) == time_steps and len(rotate_frames) == time_steps and len(zoom_frames) == time_steps:
                X.append(frames)
                y = [0]*len(classes)
                y[classes.index(c)] = 1
                Y.append(y)
                
                X.append(rotate_frames)
                y = [0]*len(classes)
                y[classes.index(c)] = 1
                Y.append(y)
                
                X.append(zoom_frames)
                y = [0]*len(classes)
                y[classes.index(c)] = 1
                Y.append(y)
                
    X = np.asarray(X)
    Y = np.asarray(Y)
    return X, Y

X, Y = create_data(data_dir)
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.5, shuffle=True)
X = np.concatenate((X_train, X_test))
Y = np.concatenate((y_train, y_test))

['Backstroke', 'Breaststroke', 'Freestyle']
Backstroke
Breaststroke
Freestyle


In [8]:
import matplotlib.pyplot as plt

print(X.shape)
#X_train = np.expand_dims(X_train, axis=-1)
#X_test = np.expand_dims(X_test, axis=-1)
#y_train = np.expand_dims(y_train, axis=-1)
#y_test = np.expand_dims(y_test, axis=-1)

#print(X_train.shape)
#img = cv2_clipped_zoom(X[2][8], 1.5)
#plt.imshow(img)

#plt.plot()

print(len(X))
print(len(Y))

(381, 10, 140, 90, 3)
381
381


In [9]:
model = Sequential()
# input_shape: (time_steps, map_height, map_width, channels)
model.add(ConvLSTM2D(filters = 32, kernel_size = (3, 3), input_shape = (time_steps, h, w, 3), return_sequences = True))
model.add(ConvLSTM2D(filters = 32, kernel_size = (3, 3), return_sequences = True))
model.add(BatchNormalization())
#model.add(LeakyReLU(alpha=0.2))
model.add(Dropout(0.2))


model.add(ConvLSTM2D(filters = 32, kernel_size = (3, 3), return_sequences = True))
model.add(ConvLSTM2D(filters = 32, kernel_size = (3, 3), return_sequences = True))
model.add(BatchNormalization())
#model.add(LeakyReLU(alpha=0.2))
model.add(MaxPooling3D((1, 2, 2)))
model.add(Dropout(0.2))

model.add(ConvLSTM2D(filters = 32, kernel_size = (3, 3), return_sequences = True))
model.add(ConvLSTM2D(filters = 32, kernel_size = (3, 3), return_sequences = True))
model.add(BatchNormalization())
#model.add(LeakyReLU(alpha=0.2))
#model.add(AveragePooling3D((1, 2, 2)))
model.add(Dropout(0.2))

model.add(ConvLSTM2D(filters = 32, kernel_size = (3, 3), return_sequences = True))
model.add(ConvLSTM2D(filters = 32, kernel_size = (3, 3), return_sequences = False))
model.add(BatchNormalization())
#model.add(LeakyReLU(alpha=0.2))
model.add(AveragePooling2D((2, 2)))
model.add(Dropout(0.2))

model.add(Flatten())
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(3, activation='softmax'))
model.summary()

opt = Adam(lr=1e-6)
#opt = SGD(lr=0.0001)

model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=["accuracy"])

checkpoint = ModelCheckpoint(filepath=filename,
                             monitor='val_loss',
                             verbose=1,
                             save_best_only=True)

stop = EarlyStopping(monitor='loss', patience = 10,
                      verbose=0, mode='auto', baseline=None, 
                      restore_best_weights=False)
callbacks = [checkpoint, stop]

history = model.fit(x = X, y = Y, epochs = 1000, batch_size = 4, shuffle=True, validation_split=0.2, callbacks=callbacks)
#history = model.fit(train, validation_data=valid, verbose=1, epochs=50, shuffle=True, callbacks=callbacks)
print("Training finished.")

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv_lst_m2d_16 (ConvLSTM2D) (None, 10, 138, 88, 32)   40448     
_________________________________________________________________
conv_lst_m2d_17 (ConvLSTM2D) (None, 10, 136, 86, 32)   73856     
_________________________________________________________________
batch_normalization_8 (Batch (None, 10, 136, 86, 32)   128       
_________________________________________________________________
dropout_9 (Dropout)          (None, 10, 136, 86, 32)   0         
_________________________________________________________________
conv_lst_m2d_18 (ConvLSTM2D) (None, 10, 134, 84, 32)   73856     
_________________________________________________________________
conv_lst_m2d_19 (ConvLSTM2D) (None, 10, 132, 82, 32)   73856     
_________________________________________________________________
batch_normalization_9 (Batch (None, 10, 132, 82, 32)  

Epoch 16/1000
76/76 [==============================] - ETA: 0s - loss: 0.8782 - accuracy: 0.5954
Epoch 00016: val_loss improved from 0.85344 to 0.84063, saving model to ConvLSTM_model.h5
76/76 [==============================] - 38s 505ms/step - loss: 0.8782 - accuracy: 0.5954 - val_loss: 0.8406 - val_accuracy: 0.5325
Epoch 17/1000
76/76 [==============================] - ETA: 0s - loss: 0.8995 - accuracy: 0.5658
Epoch 00017: val_loss improved from 0.84063 to 0.82829, saving model to ConvLSTM_model.h5
76/76 [==============================] - 40s 522ms/step - loss: 0.8995 - accuracy: 0.5658 - val_loss: 0.8283 - val_accuracy: 0.5455
Epoch 18/1000
76/76 [==============================] - ETA: 0s - loss: 0.8551 - accuracy: 0.5691
Epoch 00018: val_loss improved from 0.82829 to 0.81733, saving model to ConvLSTM_model.h5
76/76 [==============================] - 40s 520ms/step - loss: 0.8551 - accuracy: 0.5691 - val_loss: 0.8173 - val_accuracy: 0.5455
Epoch 19/1000
76/76 [======================

Epoch 42/1000
76/76 [==============================] - ETA: 0s - loss: 0.7011 - accuracy: 0.6711
Epoch 00042: val_loss improved from 0.60028 to 0.59512, saving model to ConvLSTM_model.h5
76/76 [==============================] - 40s 530ms/step - loss: 0.7011 - accuracy: 0.6711 - val_loss: 0.5951 - val_accuracy: 0.6104
Epoch 43/1000
76/76 [==============================] - ETA: 0s - loss: 0.7228 - accuracy: 0.6743
Epoch 00043: val_loss improved from 0.59512 to 0.59018, saving model to ConvLSTM_model.h5
76/76 [==============================] - 38s 506ms/step - loss: 0.7228 - accuracy: 0.6743 - val_loss: 0.5902 - val_accuracy: 0.6104
Epoch 44/1000
76/76 [==============================] - ETA: 0s - loss: 0.6475 - accuracy: 0.7039
Epoch 00044: val_loss improved from 0.59018 to 0.58441, saving model to ConvLSTM_model.h5
76/76 [==============================] - 38s 502ms/step - loss: 0.6475 - accuracy: 0.7039 - val_loss: 0.5844 - val_accuracy: 0.6104
Epoch 45/1000
76/76 [======================

Epoch 68/1000
76/76 [==============================] - ETA: 0s - loss: 0.5708 - accuracy: 0.7500
Epoch 00068: val_loss improved from 0.42320 to 0.41154, saving model to ConvLSTM_model.h5
76/76 [==============================] - 38s 493ms/step - loss: 0.5708 - accuracy: 0.7500 - val_loss: 0.4115 - val_accuracy: 0.8052
Epoch 69/1000
76/76 [==============================] - ETA: 0s - loss: 0.4615 - accuracy: 0.8289
Epoch 00069: val_loss improved from 0.41154 to 0.40102, saving model to ConvLSTM_model.h5
76/76 [==============================] - 37s 490ms/step - loss: 0.4615 - accuracy: 0.8289 - val_loss: 0.4010 - val_accuracy: 0.8052
Epoch 70/1000
76/76 [==============================] - ETA: 0s - loss: 0.4647 - accuracy: 0.8059
Epoch 00070: val_loss improved from 0.40102 to 0.38837, saving model to ConvLSTM_model.h5
76/76 [==============================] - 38s 494ms/step - loss: 0.4647 - accuracy: 0.8059 - val_loss: 0.3884 - val_accuracy: 0.7922
Epoch 71/1000
76/76 [======================

76/76 [==============================] - ETA: 0s - loss: 0.3796 - accuracy: 0.8586
Epoch 00094: val_loss improved from 0.21785 to 0.20206, saving model to ConvLSTM_model.h5
76/76 [==============================] - 38s 494ms/step - loss: 0.3796 - accuracy: 0.8586 - val_loss: 0.2021 - val_accuracy: 0.9610
Epoch 95/1000
76/76 [==============================] - ETA: 0s - loss: 0.3375 - accuracy: 0.8651
Epoch 00095: val_loss did not improve from 0.20206
76/76 [==============================] - 37s 492ms/step - loss: 0.3375 - accuracy: 0.8651 - val_loss: 0.2026 - val_accuracy: 0.9610
Epoch 96/1000
76/76 [==============================] - ETA: 0s - loss: 0.3821 - accuracy: 0.8322
Epoch 00096: val_loss improved from 0.20206 to 0.19818, saving model to ConvLSTM_model.h5
76/76 [==============================] - 37s 491ms/step - loss: 0.3821 - accuracy: 0.8322 - val_loss: 0.1982 - val_accuracy: 0.9610
Epoch 97/1000
76/76 [==============================] - ETA: 0s - loss: 0.3641 - accuracy: 0.8717

In [ ]:
#y_train = model.predict(X_train)
#y_val = model.predict(X_test)

#y_train = np.argmax(y_train, axis = 1)
#y_val = np.argmax(y_val, axis = 1)

#print("Train: "+str(y_train))
#print("Validation: "+str(y_val))